# Jupyter Notebook to Merge COVID Related Data From Multiple Data Sources 

#### _Work done by Nepal Poverty Team, The World Bank_

## Data Sources:
1. [Google Community Mobility Reports](https://www.google.com/covid19/mobility/)
2. [The Oxford COVID-19 Government Response Tracker](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker) 
3. [Our World in Data](https://ourworldindata.org/coronavirus) 
4. [World Bank's list of economies](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups)

We have used Python 3 and produced the Python 3 Jupyter notebook showing data cleaning and merging.

## Setup

Running of this notebook requires Jupyter software system. Either Jupyter notebook or Jupyter lab can be installed on the system. In addition, two additional Python packages -- pycountry and pandas -- are required.

### Jupyter Software Installation
https://jupyter.org/install

### pycountry Package Installation
https://pypi.org/project/pycountry/

### pandas Package Installation
https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html

After all the dependencies are installed the notebook can be imported to the Jupyter software and run.

## Imports

In [1]:
import time
import datetime
import requests
import pycountry
import pandas as pd

## Save the column names of latest CSV file in a list

In [2]:
# skip this cell if you are using this notebook for the first time or do not have previous version of GCMR_OWID_OxCGRT_WB_x.csv file.
latest_CSV_file = 'GCMR_OWID_OxCGRT_WB_1593014927.5399244.csv'
columns = pd.read_csv(latest_CSV_file, nrows=1).columns.tolist()
drop_columns = set(['OXCGRT_Date', 'OXCGRT_CountryCode', 'OWID_iso_code', 'OWID_date', 'GCMR_country_region_code', 'GCMR_date'])

## Data extraction from URLs

In [2]:
# getting data from the web URLs

google_url = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"
google_data = pd.read_csv(google_url)
print("Google mobility data fetched.")

owid_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
owid_data = pd.read_csv(owid_url)
print("OWID data fetched.")

oxford_url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
oxford_data = pd.read_csv(oxford_url)
print("OxCGRT data fetched.")

/home/regmi/envs/data_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Google mobility data fetched.
OWID data fetched.
OxCGRT data fetched.


/home/regmi/envs/data_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data cleaning

We saw country region code for Namibia missing in the Google mobility data, so we add the country code for Namibia. Similarly, we add `INTL` as country code for International numbers. They are the numbers not belonging to any country.

In [3]:
# Add 'NAM' as country_region_code for Namibia

google_data.loc[google_data[google_data['country_region'] == 'Namibia'].index, 'country_region_code'] = "NA"
assert google_data['country_region_code'].isnull().sum() == 0

In [4]:
# Add INTL as international ISO code for owid_data
owid_data.loc[owid_data[owid_data['location'] == 'International'].index, 'iso_code'] = 'INTL'
assert owid_data['iso_code'].isnull().sum() == 0

# also assert oxford_data does not have any null country codes
assert oxford_data['CountryCode'].isnull().sum() == 0

Let's prepend the column names by `GCMR_`, `OWID_` and `OXCGRT_` for Google mobility data, OWID data and Oxford policy tracker data respectively. This helps us to distinguish the source of the columns.

In [5]:
# GCMR for Google Community Mobility Report
google_data.columns = ['GCMR_' + i for i in google_data.columns]

# OWID for Our World in Development
owid_data.columns = ['OWID_' + i for i in owid_data.columns]

# OXCGRT for Oxford COVID-19 Government Response Tracker
oxford_data.columns = ['OXCGRT_' + i for i in oxford_data.columns]

### Column names validation

In [6]:
print("New columns:",set(google_data.columns).union(set(owid_data.columns)).union(set(oxford_data.columns)) - set(columns).union(drop_columns))

NameError: name 'columns' is not defined

In [7]:
# raise error if any of the different column name has been used in any of the data or new column has been introduced
# this should be fixed manually seeing the different or new column name
assert set(google_data.columns).union(set(owid_data.columns)).union(set(oxford_data.columns)) - set(columns).union(drop_columns) == set()

NameError: name 'columns' is not defined

In [8]:
google_data.shape, owid_data.shape, oxford_data.shape

((3816670, 14), (59745, 55), (106120, 49))

In [9]:
oxford_data = oxford_data[oxford_data['OXCGRT_RegionCode'].isnull()]

In [10]:
google_data = google_data[google_data['GCMR_sub_region_1'].isnull() & google_data['GCMR_metro_area'].isnull()]

In [11]:
google_data.shape, owid_data.shape, oxford_data.shape

((44193, 14), (59745, 55), (69736, 49))

Let's bring date of Oxford policy tracker data to the desired, compatible date format.

In [12]:
oxford_data['OXCGRT_Date'] = oxford_data['OXCGRT_Date'].apply(lambda x: str(x)).apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])

## Outer merging of Oxford data, OWID data, and Google mobility data on country and date

In [13]:
new_df = pd.merge(oxford_data, owid_data,  how='outer', left_on=['OXCGRT_CountryCode', 'OXCGRT_Date'], right_on = ['OWID_iso_code', 'OWID_date'])

In [14]:
new_df.shape

(75140, 104)

Let's create a function <i>get_a_or_b</i> which gets either <i>a</i> or <i>b</i>, depending on which value is non-null. 

In [15]:
def get_a_or_b(row, a, b):
    
    row = row.fillna('')
    
    if row[a]:
        return row[a]
    elif row[b]:
        return row[b]   

Apply <i>get_a_or_b</i> to ISO codes and dates and save them in the two columns, `iso_code` and `date`. It helps to get the non-null columns, `iso_code` and `date`.

In [16]:
new_df['iso_code'] = new_df.apply(get_a_or_b, args=('OXCGRT_CountryCode', 'OWID_iso_code'), axis=1)
new_df['date'] = new_df.apply(get_a_or_b, args=('OXCGRT_Date', 'OWID_date'), axis=1)

Now, let's delete the columns we applied the function to.

In [17]:
new_df.drop(['OXCGRT_Date', 'OXCGRT_CountryCode', 'OWID_iso_code', 'OWID_date'], axis=1, inplace=True)

We see two different set of country codes being used across the datasets -- two character and three character country codes (Example: NP and NPL for Nepal). Let's bring uniformity by converting the two character country codes to three character country codes using `pycountry` Python package.

In [18]:
google_data['GCMR_country_region_code'] = google_data['GCMR_country_region_code'].apply(lambda x: pycountry.countries.get(alpha_2=x).alpha_3)

Let's now outer merge the above merged data with google data on country code and date.

In [19]:
final_df = pd.merge(google_data, new_df,  how='outer', left_on=['GCMR_country_region_code', 'GCMR_date'], right_on = ['iso_code', 'date'])

Apply <i>get_a_or_b</i> to ISO codes and dates and assign them in the two columns, `iso_code` and `date`. It helps to get the non-null columns, `iso_code` and `date` -- same operation done as above.

In [20]:
final_df['iso_code'] = final_df.apply(get_a_or_b, args=('GCMR_country_region_code', 'iso_code'), axis=1)
final_df['date'] = final_df.apply(get_a_or_b, args=('GCMR_date', 'date'), axis=1)

Now, let's delete the columns we applied the function to.

In [21]:
final_df.drop(['GCMR_country_region_code', 'GCMR_date'], axis=1, inplace=True)

Similarly, let's have one single column `Country` which represents the country the data row belongs to. As done above, we apply the <i>get_a_or_b</i> function two times.

In [22]:
final_df['Country'] = final_df.apply(get_a_or_b, args=('GCMR_country_region', 'OXCGRT_CountryName'), axis=1)

In [23]:
final_df['Country'] = final_df.apply(get_a_or_b, args=('Country', 'OWID_location'), axis=1)

## Merge with WB's `List of Economies` data

In [24]:
economies = pd.read_excel('CLASS (1).xls', sheet_name='List of economies')

### Preprocess the data

In [25]:
economies.columns = economies.iloc[3, :]
economies = economies.iloc[5:223, :]
economies = economies[['Economy', 'Code', 'Region', 'Income group', 'Lending category', 'Other']]
economies['Lending category'] = economies['Lending category'].replace(to_replace='\.\.', value='', regex=True)

Prepend the columns with `WB_` to identify the columns coming from WB's `List of Economies` data.

In [26]:
economies.columns = ['WB_' + i for i in economies.columns]

### Merge `final_df` and WB's economies data on country codes.

In [27]:
merged_df = pd.merge(final_df, economies,  how='outer', left_on=['iso_code'], right_on = ['WB_Code'])

Apply the same function as above and assign the country name information in `Country` column.

In [28]:
merged_df['Country'] = merged_df.apply(get_a_or_b, args=('Country', 'WB_Economy'), axis=1)

## Add timestamp

In [29]:
merged_df['Timestamp (UTC)'] = datetime.datetime.utcnow().__str__()

## Export to CSV file

In [30]:
merged_df.to_csv('~/OneDrive/WB/COVID/GCMR_OWID_OxCGRT_WB_{}.csv'.format(int(time.time())), index=False)

In [45]:
merged_df[merged_df['Country'] == "Nepal"].sort_values(by=['date'], ascending=False).head(20)

,GCMR_country_region,GCMR_sub_region_1,GCMR_sub_region_2,GCMR_metro_area,GCMR_iso_3166_2_code,GCMR_census_fips_code,GCMR_retail_and_recreation_percent_change_from_baseline,GCMR_grocery_and_pharmacy_percent_change_from_baseline,GCMR_parks_percent_change_from_baseline,GCMR_transit_stations_percent_change_from_baseline,...,iso_code,date,Country,WB_Economy,WB_Code,WB_Region,WB_Income group,WB_Lending category,WB_Other,Timestamp (UTC)
30981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NPL,2020-11-26,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NPL,2020-11-25,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NPL,2020-11-24,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NPL,2020-11-23,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30932,Nepal,NaN,NaN,NaN,NaN,NaN,-22.0,1.0,-6.0,-1.0,...,NPL,2020-11-22,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30931,Nepal,NaN,NaN,NaN,NaN,NaN,-32.0,-4.0,-8.0,-9.0,...,NPL,2020-11-21,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30930,Nepal,NaN,NaN,NaN,NaN,NaN,-29.0,0.0,-4.0,-10.0,...,NPL,2020-11-20,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30929,Nepal,NaN,NaN,NaN,NaN,NaN,-26.0,2.0,-9.0,-6.0,...,NPL,2020-11-19,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30928,Nepal,NaN,NaN,NaN,NaN,NaN,-27.0,0.0,-12.0,-6.0,...,NPL,2020-11-18,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
30927,Nepal,NaN,NaN,NaN,NaN,NaN,-37.0,-11.0,-6.0,-16.0,...,NPL,2020-11-17,Nepal,Nepal,NPL,South Asia,Lower middle income,IDA,NaN,2020-11-26 08:46:54.787897
